In [1]:
pip install influxdb

     |████████████████████████████████| 77 kB 1.3 MB/s eta 0:00:011
     |████████████████████████████████| 245 kB 3.3 MB/s eta 0:00:01
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.0
    Uninstalling msgpack-1.0.0:
      Successfully uninstalled msgpack-1.0.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install country_converter --upgrade

Requirement already up-to-date: country_converter in /opt/conda/lib/python3.7/site-packages (0.6.7)
Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install geopandas

     |████████████████████████████████| 928 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 14.7 MB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 11.7 MB/s eta 0:00:01
     |████████████████████████████████| 10.9 MB 7.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import country_converter as coco

In [2]:
import pandas as pd
from influxdb import DataFrameClient

In [33]:
import geopandas

In [3]:
user = 'root'
password = 'root'
host='influxdb' #172.28.1.2 #localhost #3.14.5.146 - kozos osztaly InfluxDB szerver
port=8086
dbname='base' #az en adatbazis nevem
protocol = 'line' #json
client = DataFrameClient(host, port, user, password, dbname)

In [4]:
client.query('show databases')

ResultSet({'('databases', None)': [{'name': '_internal'}, {'name': 'base'}]})

In [5]:
data = pd.read_csv('owid-covid-data.csv')

In [7]:
data2 = pd.read_csv('time_series_covid19_deaths_global.csv')

In [8]:
data2=data2.drop(columns=['Province/State'])

In [9]:
data2=data2.rename(columns={'Country/Region': "location"})

In [10]:
data2

,location,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
0,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,0,...,106,109,115,120,122,127,132,136,153,168
1,Albania,41.153300,20.168300,0,0,0,0,0,0,0,...,31,31,31,31,31,31,31,31,31,31
2,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,...,483,488,494,502,507,515,522,529,536,542
3,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,...,47,47,48,48,48,48,49,49,49,51
4,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Sao Tome and Principe,0.186360,6.613081,0,0,0,0,0,0,0,...,4,5,5,5,5,5,6,7,7,7
262,Yemen,15.552727,48.516388,0,0,0,0,0,0,0,...,5,7,7,8,9,10,12,12,15,18
263,Comoros,-11.645500,43.333300,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
264,Tajikistan,38.861034,71.276093,0,0,0,0,0,0,0,...,12,12,20,20,21,21,23,29,33,36


In [10]:
client.write_points(df, measurement, protocol=protocol,
                        field_columns=['total_cases_per_million'],
                        tag_columns=['location'])

NameError: name 'df' is not defined

In [8]:
measurement='teszt'

In [8]:
df=pd.DataFrame(data).set_index('date')
df.index=pd.to_datetime(df.index)
df['new_deaths']=df['new_deaths'].astype(int)
df['total_deaths']=df['total_deaths'].astype(int)
df['location']=df['location'].astype(str)
client.write_points(df, 'scatter', protocol=protocol,
                        field_columns=['new_deaths', 'total_deaths', 'location'],
                        tag_columns=['total_deaths', 'location'])

True

In [11]:
df=pd.DataFrame(data).set_index('date')
df.index=pd.to_datetime(df.index)

In [7]:
df=pd.DataFrame(data).set_index('date')
df.index=pd.to_datetime(df.index)
df['total_cases']=df['total_cases'].astype(int)
df['total_deaths']=df['total_deaths'].astype(int)
df['population']=df['population'].astype(float)
client.write_points(df, 'total', protocol=protocol,
                        field_columns=['total_cases', 'total_deaths', 'population'],
                        tag_columns=['location'])

True

In [12]:
df


,iso_code,location,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
date,,,,,,,,,,,,,,,,,,,,,
2020-03-13,ABW,Aruba,2,2,0,0,18.733,18.733,0.0,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
2020-03-20,ABW,Aruba,4,2,0,0,37.465,18.733,0.0,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
2020-03-24,ABW,Aruba,12,8,0,0,112.395,74.930,0.0,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
2020-03-25,ABW,Aruba,17,5,0,0,159.227,46.831,0.0,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
2020-03-26,ABW,Aruba,19,2,0,0,177.959,18.733,0.0,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-28,NaN,International,705,0,4,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-29,NaN,International,705,0,6,2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-01,NaN,International,705,0,6,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
wm=data[data.date == '2020-05-15']

In [144]:
arr=data1['location'].to_numpy()

In [149]:
data1 = coco.convert(names=res, to='ISO2', not_found=None)


In [160]:
df1=df1[df1[0] != 'Timor']

In [139]:
data1=data[data['location'] != 'International']

In [141]:
data1=data1[data1['location'] != 'World']

In [147]:
res = [] 
for i in arr: 
    if i not in res: 
        res.append(i) 

In [161]:
df1

,0
0,AW
1,AF
2,AO
3,AI
4,AL
...,...
205,VN
206,YE
207,ZA
208,ZM


In [26]:
df.dtypes

iso_code                     object
location                     object
total_cases                   int64
new_cases                     int64
total_deaths                  int64
new_deaths                    int64
total_cases_per_million     float64
new_cases_per_million       float64
total_deaths_per_million    float64
new_deaths_per_million      float64
total_tests                 float64
new_tests                   float64
total_tests_per_thousand    float64
new_tests_per_thousand      float64
tests_units                  object
population                  float64
population_density          float64
median_age                  float64
aged_65_older               float64
aged_70_older               float64
gdp_per_capita              float64
extreme_poverty             float64
cvd_death_rate              float64
diabetes_prevalence         float64
female_smokers              float64
male_smokers                float64
handwashing_facilities      float64
hospital_beds_per_100k      

In [20]:
data.location.unique()

array(['Aruba', 'Afghanistan', 'Angola', 'Anguilla', 'Albania', 'Andorra',
       'United Arab Emirates', 'Argentina', 'Armenia',
       'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan',
       'Burundi', 'Belgium', 'Benin', 'Bonaire Sint Eustatius and Saba',
       'Burkina Faso', 'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas',
       'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda',
       'Bolivia', 'Brazil', 'Barbados', 'Brunei', 'Bhutan', 'Botswana',
       'Central African Republic', 'Canada', 'Switzerland', 'Chile',
       'China', "Cote d'Ivoire", 'Cameroon',
       'Democratic Republic of Congo', 'Congo', 'Colombia', 'Comoros',
       'Cape Verde', 'Costa Rica', 'Cuba', 'Curacao', 'Cayman Islands',
       'Cyprus', 'Czech Republic', 'Germany', 'Djibouti', 'Dominica',
       'Denmark', 'Dominican Republic', 'Algeria', 'Ecuador', 'Egypt',
       'Eritrea', 'Western Sahara', 'Spain', 'Estonia', 'Ethiopia',
       'Finland', 'Fiji', 'Falkland Islands', 'France',

In [24]:
alma=data.location.unique().replace("''", "")

AttributeError: 'numpy.ndarray' object has no attribute 'replace'

In [1]:
data


NameError: name 'data' is not defined

In [172]:
df=pd.DataFrame(data).set_index('date')
df.index=pd.to_datetime(df.index)
df['new_cases']=df['new_cases'].astype(int)
df['new_deaths']=df['new_deaths'].astype(int)
df['total_deaths']=df['total_deaths'].astype(int)
df['total_cases']=df['total_cases'].astype(int)
client.write_points(df, 'scattervegso', protocol=protocol,
                        field_columns=['new_cases', 'total_cases', 'new_deaths', 'total_deaths'],
                        tag_columns=['location', 'new_cases', 'total_cases', 'new_deaths', 'total_deaths'])

True

In [68]:
data

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
0,ABW,Aruba,2020-03-13,2,2,0,0,18.733,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
1,ABW,Aruba,2020-03-20,4,2,0,0,37.465,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
2,ABW,Aruba,2020-03-24,12,8,0,0,112.395,74.930,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
3,ABW,Aruba,2020-03-25,17,5,0,0,159.227,46.831,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
4,ABW,Aruba,2020-03-26,19,2,0,0,177.959,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17420,NaN,International,2020-02-28,705,0,4,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17421,NaN,International,2020-02-29,705,0,6,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17422,NaN,International,2020-03-01,705,0,6,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17423,NaN,International,2020-03-02,705,0,6,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
alma=data[['iso_code','location', 'date', 'total_cases', 'total_deaths', 'total_tests']]

In [29]:
alma.rename(columns={ alma.columns[3]: "total_cases" }, inplace = True)

AttributeError: 'Series' object has no attribute 'columns'

In [17]:
alma=alma[alma['location']!= 'World']

##### df.head(15)

In [24]:
df.head(150)

,iso_code,location,total_cases,total_deaths,total_tests
date,,,,,
2020-03-13,ABW,Aruba,2,0,NaN
2020-03-20,ABW,Aruba,4,0,NaN
2020-03-24,ABW,Aruba,12,0,NaN
2020-03-25,ABW,Aruba,17,0,NaN
2020-03-26,ABW,Aruba,19,0,NaN
...,...,...,...,...,...
2020-04-10,AFG,Afghanistan,484,15,NaN
2020-04-11,AFG,Afghanistan,521,15,NaN
2020-04-12,AFG,Afghanistan,555,18,NaN


In [19]:
df=pd.DataFrame(data2).set_index('date')
df.index=pd.to_datetime(df.index)

In [25]:
df['total_cases']=df['total_cases'].astype(int)
df['total_deaths']=df['total_deaths'].astype(int)
df['total_tests']=df['total_tests'].astype(float)
client.write_points(df, 'world1', protocol=protocol,
                        field_columns=['total_cases', 'total_tests', 'total_deaths'],
                        tag_columns=['iso_code', 'location'])

True

In [171]:
data2

,location,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,total_deaths
0,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,0,...,106,109,115,120,122,127,132,136,153,2523
1,Albania,41.153300,20.168300,0,0,0,0,0,0,0,...,31,31,31,31,31,31,31,31,31,1284
2,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,...,483,488,494,502,507,515,522,529,536,16883
3,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,...,47,47,48,48,48,48,49,49,49,1658
4,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Sao Tome and Principe,0.186360,6.613081,0,0,0,0,0,0,0,...,4,5,5,5,5,5,6,7,7,61
262,Yemen,15.552727,48.516388,0,0,0,0,0,0,0,...,5,7,7,8,9,10,12,12,15,104
263,Comoros,-11.645500,43.333300,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,10
264,Tajikistan,38.861034,71.276093,0,0,0,0,0,0,0,...,12,12,20,20,21,21,23,29,33,211


In [178]:
alma

,location,date,total_cases,total_deaths,total_tests
0,Aruba,2020-03-13,2,0,NaN
1,Aruba,2020-03-20,4,0,NaN
2,Aruba,2020-03-24,12,0,NaN
3,Aruba,2020-03-25,17,0,NaN
4,Aruba,2020-03-26,19,0,NaN
...,...,...,...,...,...
17219,Zimbabwe,2020-05-11,36,4,10598.0
17220,Zimbabwe,2020-05-12,37,4,11058.0
17221,Zimbabwe,2020-05-13,37,4,11191.0
17222,Zimbabwe,2020-05-14,37,4,NaN


In [177]:
df1.rename(columns={0: 'iso_2'})

,iso_2
0,AW
1,AF
2,AO
3,AI
4,AL
...,...
204,VN
205,YE
206,ZA
207,ZM


In [176]:
df1=df1.reset_index()

In [201]:

df1

,0
0,AW
1,AF
2,AO
3,AI
4,AL
...,...
204,VN
205,YE
206,ZA
207,ZM


In [221]:
alma=alma.rename(columns={0: 'iso_2'})

In [41]:
korte=alma

In [40]:
korte.drop(ABW)

,iso_code,location,date,total_cases,total_deaths,total_tests
0,ABW,Aruba,2020-03-13,2,0,NaN
1,ABW,Aruba,2020-03-20,4,0,NaN
2,ABW,Aruba,2020-03-24,12,0,NaN
3,ABW,Aruba,2020-03-25,17,0,NaN
4,ABW,Aruba,2020-03-26,19,0,NaN
5,ABW,Aruba,2020-03-27,28,0,NaN
6,ABW,Aruba,2020-03-28,28,0,NaN
7,ABW,Aruba,2020-03-29,28,0,NaN
8,ABW,Aruba,2020-03-30,50,0,NaN
9,ABW,Aruba,2020-04-01,55,0,NaN


In [44]:
data2

,location,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
0,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,0,...,106,109,115,120,122,127,132,136,153,168
1,Albania,41.153300,20.168300,0,0,0,0,0,0,0,...,31,31,31,31,31,31,31,31,31,31
2,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,...,483,488,494,502,507,515,522,529,536,542
3,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,...,47,47,48,48,48,48,49,49,49,51
4,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Sao Tome and Principe,0.186360,6.613081,0,0,0,0,0,0,0,...,4,5,5,5,5,5,6,7,7,7
262,Yemen,15.552727,48.516388,0,0,0,0,0,0,0,...,5,7,7,8,9,10,12,12,15,18
263,Comoros,-11.645500,43.333300,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
264,Tajikistan,38.861034,71.276093,0,0,0,0,0,0,0,...,12,12,20,20,21,21,23,29,33,36


In [57]:
korte['Lat'] = korte['iso_code']

In [46]:
korte = korte[korte.iso_code != 'ABW']

In [80]:
korte=korte.replace('33.000000', '65.000000')

In [74]:
korte=korte.drop(columns=['Lat'])

In [125]:
orszag=korte['location'].tolist()

In [126]:
lon=data2['Long'].tolist()

In [135]:
len(data2.columns)


119

In [153]:
data2

,location,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,total_deaths
0,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,0,...,109,115,120,122,127,132,136,153,168,168
1,Albania,41.153300,20.168300,0,0,0,0,0,0,0,...,31,31,31,31,31,31,31,31,31,31
2,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,...,488,494,502,507,515,522,529,536,542,542
3,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,...,47,48,48,48,48,49,49,49,51,51
4,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Sao Tome and Principe,0.186360,6.613081,0,0,0,0,0,0,0,...,5,5,5,5,5,6,7,7,7,7
262,Yemen,15.552727,48.516388,0,0,0,0,0,0,0,...,7,7,8,9,10,12,12,15,18,18
263,Comoros,-11.645500,43.333300,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
264,Tajikistan,38.861034,71.276093,0,0,0,0,0,0,0,...,12,20,20,21,21,23,29,33,36,36


In [160]:
data2

,location,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
0,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,0,...,106,109,115,120,122,127,132,136,153,168
1,Albania,41.153300,20.168300,0,0,0,0,0,0,0,...,31,31,31,31,31,31,31,31,31,31
2,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,...,483,488,494,502,507,515,522,529,536,542
3,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,...,47,47,48,48,48,48,49,49,49,51
4,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Sao Tome and Principe,0.186360,6.613081,0,0,0,0,0,0,0,...,4,5,5,5,5,5,6,7,7,7
262,Yemen,15.552727,48.516388,0,0,0,0,0,0,0,...,5,7,7,8,9,10,12,12,15,18
263,Comoros,-11.645500,43.333300,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
264,Tajikistan,38.861034,71.276093,0,0,0,0,0,0,0,...,12,12,20,20,21,21,23,29,33,36


In [118]:
orszlen = len(orszag) 
lonlen=len(lon)
for j in range(lonlen):
    while
        if(orszag[i] == orszag[i+1]):
            orszag[i]=lon[j]

In [169]:
data2=data2.drop(['5/16/20'],axis=1)

In [152]:
data2['total_deaths']=data2.iloc[:, lambda df: [3, 118]].sum(axis=1)

In [167]:
data2['total_deaths']= data2.iloc[:, -116:-1].sum(axis=1)

In [170]:
data2

,location,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,total_deaths
0,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,0,...,106,109,115,120,122,127,132,136,153,2523
1,Albania,41.153300,20.168300,0,0,0,0,0,0,0,...,31,31,31,31,31,31,31,31,31,1284
2,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,...,483,488,494,502,507,515,522,529,536,16883
3,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,...,47,47,48,48,48,48,49,49,49,1658
4,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Sao Tome and Principe,0.186360,6.613081,0,0,0,0,0,0,0,...,4,5,5,5,5,5,6,7,7,61
262,Yemen,15.552727,48.516388,0,0,0,0,0,0,0,...,5,7,7,8,9,10,12,12,15,104
263,Comoros,-11.645500,43.333300,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,10
264,Tajikistan,38.861034,71.276093,0,0,0,0,0,0,0,...,12,12,20,20,21,21,23,29,33,211
